In [1]:
import pandas as pd
import pyarrow.parquet as pq
import os

# =============================================================================
# USER CONFIGURATION
# =============================================================================
GENE_NAME = "CBX8"
GENE_ID = 'ENSG00000141570.11'
PARQUET_PATH = r"C:\Users\mgcal\OneDrive\Documents\School\Research\Genetics\cTWAS Generalization\Code\eQTL_annotations_for_susine\data\gtex\GTEx_Analysis_v10_QTLs_GTEx_Analysis_v10_eQTL_all_associations_Lung.v10.allpairs.chr17.parquet"

# Filtering Parameters
MAF_MIN = 0.01
MAF_MAX = 0.99
MIN_SAMPLE_SIZE = 50

# Borzoi Model Parameters (standard values)
SEQ_LEN = 524288
LABEL_LEN = 196608

# =============================================================================
# LOAD DATA
# =============================================================================

# Read parquet with filter to only load gene data
print(f"Loading data for {GENE_NAME} ({GENE_ID})...")
table = pq.read_table(
    PARQUET_PATH,
    filters=[('gene_id', '==', GENE_ID)]
)
df = table.to_pandas()

print(f"Loaded {len(df)} SNPs")
print(f"Columns: {df.columns.tolist()}")
df.head()

Loading data for CBX8 (ENSG00000141570.11)...
Loaded 9791 SNPs
Columns: ['gene_id', 'variant_id', 'tss_distance', 'af', 'ma_samples', 'ma_count', 'pval_nominal', 'slope', 'slope_se']


,gene_id,variant_id,tss_distance,af,ma_samples,ma_count,pval_nominal,slope,slope_se
0,ENSG00000141570.11,chr17_78802073_C_T_b38,-999610,0.481450,438,578,0.216643,-0.057547,0.046522
1,ENSG00000141570.11,chr17_78802280_C_T_b38,-999403,0.490614,444,589,0.179459,-0.062319,0.046362
2,ENSG00000141570.11,chr17_78803713_G_A_b38,-997970,0.494176,447,594,0.341391,-0.043995,0.046200
3,ENSG00000141570.11,chr17_78803778_T_C_b38,-997905,0.548253,414,543,0.946131,0.003089,0.045701
4,ENSG00000141570.11,chr17_78803816_A_G_b38,-997867,0.545757,414,546,0.889546,0.006307,0.045394


In [2]:
# Add sample_size column
df['sample_size'] = df['ma_count'] / (2 * df['af'])

# =============================================================================
# APPLY FILTERS
# =============================================================================
print(f"Filtering variants...")
print(f"  Initial count: {len(df)}")

# MAF Filter
df = df[(df['af'] >= MAF_MIN) & (df['af'] <= MAF_MAX)]
print(f"  After MAF filter ({MAF_MIN} <= AF <= {MAF_MAX}): {len(df)}")

# Sample Size Filter
df = df[df['sample_size'] > MIN_SAMPLE_SIZE]
print(f"  After Sample Size filter (>{MIN_SAMPLE_SIZE}): {len(df)}")

# =============================================================================
# STATISTICS
# =============================================================================

# Define thresholds in base pairs
tss_threshold_100kb = 100_000
tss_threshold_1mb = 1_000_000

# Calculate statistics for 100KB radius
snps_within_100kb = len(df[df['tss_distance'].abs() <= tss_threshold_100kb])
snps_outside_100kb = len(df[df['tss_distance'].abs() > tss_threshold_100kb])
sig_snps_within_100kb = len(df[(df['tss_distance'].abs() <= tss_threshold_100kb) & (df['pval_nominal'] < 0.05)])
sig_snps_outside_100kb = len(df[(df['tss_distance'].abs() > tss_threshold_100kb) & (df['pval_nominal'] < 0.05)])

# Calculate statistics for 1MB radius
total_snps = len(df)
snps_within_1mb = len(df[df['tss_distance'].abs() <= tss_threshold_1mb])
snps_outside_1mb = total_snps - snps_within_1mb
sig_snps_within_1mb = len(df[(df['tss_distance'].abs() <= tss_threshold_1mb) & (df['pval_nominal'] < 0.05)])
sig_snps_outside_1mb = len(df[(df['tss_distance'].abs() > tss_threshold_1mb) & (df['pval_nominal'] < 0.05)])

# Calculate average effective sample sizes
avg_sample_size_all = df['sample_size'].mean()
avg_sample_size_100kb = df[df['tss_distance'].abs() <= tss_threshold_100kb]['sample_size'].mean()
avg_sample_size_1mb = df[df['tss_distance'].abs() <= tss_threshold_1mb]['sample_size'].mean()

print(f"\n=== {GENE_NAME} eQTL Analysis (Filtered) ===\n")
print(f"Total SNPs: {total_snps}")
print(f"Average effective sample size (all SNPs): {avg_sample_size_all:.1f}")

print(f"\n--- 100KB Radius ---")
print(f"SNPs within 100KB of TSS: {snps_within_100kb}")
print(f"SNPs outside 100KB of TSS: {snps_outside_100kb}")
print(f"Significant SNPs (pval_nominal < 0.05):")
print(f"  - Within 100KB of TSS: {sig_snps_within_100kb}")
print(f"  - Outside 100KB of TSS: {sig_snps_outside_100kb}")
print(f"Significance rate within 100KB: {sig_snps_within_100kb/snps_within_100kb*100:.1f}%")
print(f"Average effective sample size (within 100KB): {avg_sample_size_100kb:.1f}")

print(f"\n--- 1MB Radius ---")
print(f"SNPs within 1MB of TSS: {snps_within_1mb}")
print(f"SNPs outside 1MB of TSS: {snps_outside_1mb}")
print(f"Significant SNPs (pval_nominal < 0.05):")
print(f"  - Within 1MB of TSS: {sig_snps_within_1mb}")
print(f"  - Outside 1MB of TSS: {sig_snps_outside_1mb}")
print(f"Significance rate within 1MB: {sig_snps_within_1mb/snps_within_1mb*100:.1f}%")
print(f"Average effective sample size (within 1MB): {avg_sample_size_1mb:.1f}")

Filtering variants...
  Initial count: 9791
  After MAF filter (0.01 <= AF <= 0.99): 9466
  After Sample Size filter (>50): 9224

=== CBX8 eQTL Analysis (Filtered) ===

Total SNPs: 9224
Average effective sample size (all SNPs): 550.9

--- 100KB Radius ---
SNPs within 100KB of TSS: 788
SNPs outside 100KB of TSS: 8436
Significant SNPs (pval_nominal < 0.05):
  - Within 100KB of TSS: 292
  - Outside 100KB of TSS: 514
Significance rate within 100KB: 37.1%
Average effective sample size (within 100KB): 554.0

--- 1MB Radius ---
SNPs within 1MB of TSS: 9224
SNPs outside 1MB of TSS: 0
Significant SNPs (pval_nominal < 0.05):
  - Within 1MB of TSS: 806
  - Outside 1MB of TSS: 0
Significance rate within 1MB: 8.7%
Average effective sample size (within 1MB): 550.9


In [3]:
import sys
import os
import gzip
import urllib.request
from pathlib import Path
from typing import Dict, Tuple
import pandas as pd
import numpy as np

# =============================================================================
# UTILITY FUNCTIONS (Copied from utils/data_ingestion.py to avoid torch dependency)
# =============================================================================

def ensure_dir(path: Path) -> Path:
    """Create a directory if it does not exist and return the Path."""
    path.mkdir(parents=True, exist_ok=True)
    return path

def download_gtf_if_needed(cache_dir: Path, genome: str = "hg38") -> Path:
    """Download the GENCODE GTF file if it is not already available locally."""
    cache_dir = ensure_dir(Path(cache_dir))
    gtf_path = cache_dir / f"{genome}_gencode.gtf.gz"

    if gtf_path.exists():
        print(f"GTF file already exists: {gtf_path}")
        return gtf_path

    print("Downloading GENCODE GTF file...")
    url = "https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_44/gencode.v44.annotation.gtf.gz"
    urllib.request.urlretrieve(url, gtf_path)
    print(f"Downloaded GTF to: {gtf_path}")
    return gtf_path

def _parse_gtf_for_gene(gtf_path: Path, gene_name: str) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Stream through the GTF and retain only entries for the requested gene."""
    genes = []
    exons = []
    gene_key = gene_name.upper()
    opener = gzip.open if str(gtf_path).endswith(".gz") else open

    with opener(gtf_path, "rt") as handle:
        for line in handle:
            if not line or line.startswith("#"):
                continue

            fields = line.strip().split("\t")
            if len(fields) < 9:
                continue

            chrom, source, feature, start, end, score, strand, frame, attributes = fields

            attr_dict: Dict[str, str] = {}
            for attr in attributes.split(";"):
                attr = attr.strip()
                if not attr:
                    continue
                parts = attr.split(" ", 1)
                if len(parts) == 2:
                    key, value = parts
                    attr_dict[key] = value.strip('"')

            attr_gene_name = attr_dict.get("gene_name", "")
            if attr_gene_name.upper() != gene_key:
                continue

            if feature == "gene":
                genes.append({
                    "chrom": chrom,
                    "start": int(start) - 1,
                    "end": int(end),
                    "gene_name": attr_gene_name,
                    "gene_type": attr_dict.get("gene_type", ""),
                    "strand": strand,
                })
            elif feature == "exon":
                exons.append({
                    "chrom": chrom,
                    "start": int(start) - 1,
                    "end": int(end),
                    "gene_name": attr_gene_name,
                    "strand": strand,
                })

    genes_df = pd.DataFrame(genes)
    exons_df = pd.DataFrame(exons)

    if genes_df.empty:
        raise ValueError(f"Gene '{gene_name}' not found in annotations at {gtf_path}")

    genes_df["tss"] = genes_df.apply(
        lambda row: row["start"] if row["strand"] == "+" else row["end"] - 1,
        axis=1,
    )
    return genes_df, exons_df

def load_gene_annotations_for_gene(gtf_path: Path, gene_name: str, shortcut_dir: Path):
    """Load gene annotations for a single gene, preferring cached shortcuts."""
    shortcut_dir = ensure_dir(Path(shortcut_dir))
    gene_file = shortcut_dir / f"{gene_name}_gene_annotations.csv"
    exon_file = shortcut_dir / f"{gene_name}_exon_annotations.csv"

    if gene_file.exists() and exon_file.exists():
        print(f"Loading cached annotations for {gene_name} from {shortcut_dir}")
        genes_df = pd.read_csv(gene_file)
        exons_df = pd.read_csv(exon_file)
        return genes_df, exons_df, True

    print(f"No cached annotations for {gene_name}. Parsing full GTF...")
    genes_df, exons_df = _parse_gtf_for_gene(gtf_path, gene_name)

    genes_df.to_csv(gene_file, index=False)
    exons_df.to_csv(exon_file, index=False)
    print(f"Saved shortcuts: {gene_file.name}, {exon_file.name}")
    return genes_df, exons_df, False

def get_gene_info(gene_name: str, genes_df: pd.DataFrame, exons_df: pd.DataFrame) -> Dict:
    """Extract gene metadata and exon coordinates for the requested gene."""
    mask = genes_df["gene_name"].str.upper() == gene_name.upper()
    if not mask.any():
        raise ValueError(f"Gene '{gene_name}' not found in provided annotations")

    gene_info = genes_df[mask].iloc[0]
    gene_exons = (
        exons_df[exons_df["gene_name"].str.upper() == gene_name.upper()]
        .drop_duplicates(subset=["chrom", "start", "end"])
        .reset_index(drop=True)
    )

    result = {
        "gene_name": gene_info["gene_name"],
        "chrom": gene_info["chrom"],
        "start": int(gene_info["start"]),
        "end": int(gene_info["end"]),
        "strand": gene_info["strand"],
        "tss": int(gene_info["tss"]),
        "exons": gene_exons[["chrom", "start", "end"]].copy(),
    }
    return result

# =============================================================================
# MAIN ANALYSIS
# =============================================================================

# Borzoi Model Parameters
SEQ_LEN = 524288
LABEL_LEN = 196608

# Paths
GTF_CACHE_DIR = Path("../data/gtf_cache")
GTF_SHORTCUT_DIR = Path("../data/gtf_shortcuts")
REFERENCE_GENOME = "hg38"

# Load Gene Info
print("Loading gene annotations...")
gtf_path = download_gtf_if_needed(GTF_CACHE_DIR, genome=REFERENCE_GENOME)
genes_df, exons_df, _ = load_gene_annotations_for_gene(
    gtf_path,
    GENE_NAME,
    GTF_SHORTCUT_DIR,
)
gene_info = get_gene_info(GENE_NAME, genes_df, exons_df)
gene_start = gene_info['start']
gene_end = gene_info['end']
gene_tss = gene_info['tss']
gene_len = gene_end - gene_start

print(f"Gene: {GENE_NAME}")
print(f"  Interval: {gene_start:,} - {gene_end:,} ({gene_len:,} bp)")
print(f"  TSS: {gene_tss:,}")

# Calculate SNP positions
# tss_distance = snp_pos - tss  => snp_pos = tss + tss_distance
df['snp_pos'] = gene_tss + df['tss_distance']

# 1. TSS-centered Feasibility
# SNP must be within input window radius
limit_tss_centered = SEQ_LEN // 2
df['valid_tss_centered'] = df['tss_distance'].abs() <= limit_tss_centered

# 2. SNP-centered Feasibility
# At least 50% of gene interval must be covered in output window
output_radius = LABEL_LEN // 2

def check_snp_centered(row):
    snp_pos = row['snp_pos']
    window_start = snp_pos - output_radius
    window_end = snp_pos + output_radius
    
    overlap_start = max(gene_start, window_start)
    overlap_end = min(gene_end, window_end)
    
    overlap_len = max(0, overlap_end - overlap_start)
    return overlap_len >= (0.5 * gene_len)

df['valid_snp_centered'] = df.apply(check_snp_centered, axis=1)
df['valid_any'] = df['valid_tss_centered'] | df['valid_snp_centered']

# Report counts
n_both = len(df[df['valid_tss_centered'] & df['valid_snp_centered']])
n_tss_only = len(df[df['valid_tss_centered'] & ~df['valid_snp_centered']])
n_snp_only = len(df[~df['valid_tss_centered'] & df['valid_snp_centered']])
n_neither = len(df[~df['valid_tss_centered'] & ~df['valid_snp_centered']])

print(f"\nFeasibility Report (Total SNPs: {len(df)}):")
print(f"  Both methods feasible:       {n_both}")
print(f"  Only TSS-centered feasible:  {n_tss_only}")
print(f"  Only SNP-centered feasible:  {n_snp_only}")
print(f"  Neither feasible:            {n_neither}")

# Significant SNPs in feasible region
sig_snps = df[df['pval_nominal'] < 0.05]
sig_snps_feasible = sig_snps[sig_snps['valid_any']]
pct_sig_feasible = len(sig_snps_feasible) / len(sig_snps) * 100 if len(sig_snps) > 0 else 0

print(f"\nSignificant SNPs (pval < 0.05): {len(sig_snps)}")
print(f"Significant SNPs in Borzoi feasible region: {len(sig_snps_feasible)} ({pct_sig_feasible:.1f}%)")

# Filter for VCF (at least one feasible)
df_vcf = df[df['valid_any']].copy()
print(f"\nGenerating VCF for {len(df_vcf)} variants (at least one method feasible)...")

# Calculate Z-score
df_vcf['z_score'] = df_vcf['slope'] / df_vcf['slope_se']

# Parse variant_id to VCF columns
# Format: chr17_114226_A_G_b38
def parse_variant_id(vid):
    parts = vid.split('_')
    chrom = parts[0]
    pos = parts[1]
    ref = parts[2]
    alt = parts[3]
    return chrom, pos, ref, alt

# Apply parsing
vcf_data = df_vcf['variant_id'].apply(parse_variant_id).tolist()
vcf_df = pd.DataFrame(vcf_data, columns=['#CHROM', 'POS', 'REF', 'ALT'])
vcf_df['ID'] = df_vcf['variant_id'].values
vcf_df['QUAL'] = '.'
vcf_df['FILTER'] = 'PASS'
vcf_df['INFO'] = '.'

# Reorder columns for VCF standard
vcf_df = vcf_df[['#CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO']]

# Define output path (relative to notebook location in vignettes/)
# We want to go up one level to root, then into output/
output_dir = os.path.abspath(os.path.join(os.getcwd(), "..", "output"))
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
vcf_path = os.path.join(output_dir, f"{GENE_NAME}_variants.vcf")
csv_path = os.path.join(output_dir, f"{GENE_NAME}_GTEx_z_scores.csv")

# Write VCF header and data
with open(vcf_path, 'w') as f:
    f.write("##fileformat=VCFv4.2\n")
    f.write(f"##source=GTEx_Analysis_v10_eQTL\n")
    f.write(f"##reference=hg38\n")
    f.write("#CHROM\tPOS\tID\tREF\tALT\tQUAL\tFILTER\tINFO\n")

# Append dataframe
vcf_df.to_csv(vcf_path, mode='a', sep='\t', index=False, header=False)
print(f"VCF saved to: {vcf_path}")

# Save matching CSV with full info
df_vcf.to_csv(csv_path, index=False)
print(f"GTEx Z-scores CSV saved to: {csv_path}")

# Display first few rows
df_vcf[['variant_id', 'slope', 'slope_se', 'z_score']].head()

Loading gene annotations...
GTF file already exists: ..\data\gtf_cache\hg38_gencode.gtf.gz
Loading cached annotations for CBX8 from ..\data\gtf_shortcuts
Gene: CBX8
  Interval: 79,792,131 - 79,801,683 (9,552 bp)
  TSS: 79,801,682

Feasibility Report (Total SNPs: 9224):
  Both methods feasible:       757
  Only TSS-centered feasible:  1501
  Only SNP-centered feasible:  0
  Neither feasible:            6966

Significant SNPs (pval < 0.05): 806
Significant SNPs in Borzoi feasible region: 406 (50.4%)

Generating VCF for 2258 variants (at least one method feasible)...
VCF saved to: c:\Users\mgcal\OneDrive\Documents\School\Research\Genetics\cTWAS Generalization\Code\eQTL_annotations_for_susine\output\CBX8_variants.vcf
GTEx Z-scores CSV saved to: c:\Users\mgcal\OneDrive\Documents\School\Research\Genetics\cTWAS Generalization\Code\eQTL_annotations_for_susine\output\CBX8_GTEx_z_scores.csv


,variant_id,slope,slope_se,z_score
4088,chr17_79540699_G_T_b38,0.194860,0.154133,1.264234
4089,chr17_79540864_C_G_b38,0.007758,0.061858,0.125420
4090,chr17_79541188_C_T_b38,0.026097,0.061287,0.425817
4091,chr17_79541336_A_G_b38,0.048756,0.053850,0.905397
4092,chr17_79541515_C_G_b38,0.153977,0.150052,1.026154
